In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
import joblib
import quantstats as qs

# Configurazione grafica
plt.style.use('seaborn')
%matplotlib inline

AttributeError: 'ZMQInteractiveShell' object has no attribute 'magic'

In [ ]:
# Carica modello addestrato
model = joblib.load('models/arima_best.pkl')

# Carica dati di test
test_data = pd.read_csv('data/processed/test_data.csv', 
                       parse_dates=['Date'], 
                       index_col='Date')

In [ ]:
print(f"Ordine ARIMA: {model.order}")
print(f"Parametri: \n{model.params}")

In [ ]:
# Grafico dei residui
model.plot_diagnostics(figsize=(15,10))
plt.suptitle('Analisi Diagnostica Residui', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Genera previsioni
forecast = model.get_forecast(steps=len(test_data))
preds = forecast.predicted_mean
conf_int = forecast.conf_int()

# Plot comparativo
plt.figure(figsize=(14,6))
plt.plot(test_data, label='Valori Reali', color='#2c7bb6')
plt.plot(preds, label='Forecast', color='#d7191c', linestyle='--')
plt.fill_between(conf_int.index, 
                 conf_int.iloc[:,0], 
                 conf_int.iloc[:,1], 
                 color='#fdae61', alpha=0.3)
plt.title('Confronto Previsioni vs Valori Attuali')
plt.xlabel('Data')
plt.ylabel('Prezzo')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Calcola metriche
mape = np.mean(np.abs((test_data.values - preds.values)/test_data.values)) * 100
rmse = np.sqrt(np.mean((test_data.values - preds.values)**2))

print(f"MAPE: {mape:.2f}%")
print(f"RMSE: {rmse:.2f}")
print(f"Intervallo di Confidenza Medio: ±{np.mean(conf_int.diff(axis=1).values):.2f}")

# Report QuantStats
qs.extend_pandas()
returns = test_data.pct_change().dropna()
pred_returns = preds.pct_change().dropna()
qs.reports.html(returns, pred_returns, 
               output='reports/performance.html')

In [ ]:
errors = test_data.values - preds.values
plt.figure(figsize=(12,5))
plt.plot(errors)
plt.title('Analisi Errori nel Tempo')
plt.xlabel('Osservazioni')
plt.ylabel('Errore')
plt.hlines(0, xmin=0, xmax=len(errors), colors='red')
plt.show()